In [1]:
import openslide
import pandas as pd

In [2]:
data_dir = './datasets/brca-psj-path/ndpi'
slide = openslide.open_slide(f"{data_dir}/fffe7439-76cd-4335-9f54-d6a1992708f8.ndpi")
#display(slide.get_thumbnail((256,256)))

In [7]:
#display(slide.read_region((0,0), 5, slide.level_dimensions[5]))
slide.read_region((0,0), 5, slide.level_dimensions[5]).convert("RGB").save('slide1.jpg')

In [ ]:
import openslide, os, shutil
import pandas as pd

slides_df = pd.read_csv('./datasets/brca-psj-path/contest-phase-2/slide-manifest-train.csv')
outcomes = pd.read_csv('./datasets/brca-psj-path/contest-phase-2/csv-train/outcomes.csv')
test_df = slides_df.merge(outcomes[['biopsy_id','patient_ngsci_id','stage']], on="biopsy_id", how="left")
paths = test_df.slide_path.to_list()

for path in paths[8000:]:
    slide = openslide.open_slide(path)
    slide.read_region((0,0), 4, slide.level_dimensions[4]).convert("RGB").save(f"./slide_images/{path[40:-5]}.jpg")

images = os.listdir('./slide_images')

for image in images:
    stage = test_df[test_df.slide_id==image[:-4]].reset_index().stage[0]
    shutil.move(f"/home/ngsci/slide_images/{image}", f"/home/ngsci/slide_images/{stage}/{image}")

images = os.listdir('./datasets/brca-psj-path/contest-phase-2/clam-preprocessing-train/stitches')   
    
# for image in images:
#     stage = test_df[test_df.slide_id==image[:-4]].reset_index().stage[0]
#     shutil.copy(f"/home/ngsci/datasets/brca-psj-path/contest-phase-2/clam-preprocessing-train/stitches/{image}", f"/home/ngsci/stitches_images/{stage}/{image}")

# mv -v ~/stitches_images/IA/* ~/stitches_images/I
# mv -v ~/stitches_images/IB/* ~/stitches_images/I
# mv -v ~/stitches_images/IIA/* ~/stitches_images/II
# mv -v ~/stitches_images/IIB/* ~/stitches_images/II
# mv -v ~/stitches_images/IIIA/* ~/stitches_images/III
# mv -v ~/stitches_images/IIIB/* ~/stitches_images/III
# mv -v ~/stitches_images/IIIC/* ~/stitches_images/III
    
    
# Holdout
images = os.listdir('./datasets/brca-psj-path/ndpi-holdout')

for image in images[12000:]:
    slide = openslide.open_slide(f"./datasets/brca-psj-path/ndpi-holdout/{image}")
    slide.read_region((0,0), 4, slide.level_dimensions[4]).convert("RGB").save(f"./slide_images/holdout/{image[:-5]}.jpg")
    
# Spiltting datasets (done for each class)
class_IV = os.listdir('./stitch_images/train/IV')
for image in range(int(0.15*len(class_IV))):
    shutil.move(f"/home/ngsci/stitch_images/train/IV/{class_IV[image]}", f"/home/ngsci/stitch_images/val/IV/{class_IV[image]}")
      

In [ ]:
pixel_values[0][i][z][0].item()

In [ ]:
#must install datasets from source:
# cd datasets-main
# pip install -e .
# cd ..
# export TF_ENABLE_ONEDNN_OPTS=0
from datasets import load_dataset
dataset = load_dataset('imagefolder', data_dir='./stitch_images')

dataset['train'].features
id2label = {id:label for id, label in enumerate(dataset['train'].features['label'].names)}
label2id = {label:id for id,label in id2label.items()}
id2label

# from transformers import ViTImageProcessor, ViTForImageClassification

# from transformers import ConvNextImageProcessor, ConvNextForImageClassification

# from transformers import AutoImageProcessor, ResNetForImageClassification

from transformers import CLIPProcessor, CLIPModel

processor = CLIPProcessor.from_pretrained("clip-vit-large-patch14")
model = CLIPModel.from_pretrained("clip-vit-large-patch14", num_labels=5, ignore_mismatched_sizes=True)


In [ ]:
from torchvision.transforms import (CenterCrop, 
                                    Compose, 
                                    Normalize, 
                                    RandomHorizontalFlip,
                                    RandomResizedCrop, 
                                    Resize, 
                                    ToTensor)

image_mean, image_std = processor.image_mean, processor.image_std
# size = processor.size["height"]
size = 224

normalize = Normalize(mean=image_mean, std=image_std)
_train_transforms = Compose(
        [
            RandomResizedCrop(size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

_val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(size),
            ToTensor(),
            normalize,
        ]
    )

def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

dataset['train'].set_transform(train_transforms)
dataset['validation'].set_transform(val_transforms)

In [ ]:
from torch.utils.data import DataLoader
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return dict(accuracy=accuracy_score(predictions, labels))

from transformers import TrainingArguments, Trainer

# args = TrainingArguments(
#     'vitb16_tuned',
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=10,
#     per_device_eval_batch_size=4,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     remove_unused_columns=False,
# )

metric_name = "accuracy"

args = TrainingArguments(
    f"clip_vit_tuned",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='logs',
    remove_unused_columns=False,
)


trainer = Trainer(
    model,
    args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor
)

trainer.train()



In [ ]:
#Inference
# !export TF_ENABLE_ONEDNN_OPTS=0
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import pandas as pd
import os, torch

from datasets import load_dataset
dataset = load_dataset('imagefolder', data_dir='./stitch_images')

dataset['train'].features
id2label = {id:label for id, label in enumerate(dataset['train'].features['label'].names)}
label2id = {label:id for id,label in id2label.items()}
id2label

processor = ViTImageProcessor.from_pretrained('vitb16_tuned/checkpoint-970')
model = ViTForImageClassification.from_pretrained('vitb16_tuned/checkpoint-970')

# from transformers import ConvNextImageProcessor, ConvNextForImageClassification

# processor = ConvNextImageProcessor.from_pretrained("convnext_tuned_masks/checkpoint-1130")
# model = ConvNextForImageClassification.from_pretrained("convnext_tuned_masks/checkpoint-1130")

model.config.id2label = id2label
model.config.label2id = label2id

outputs = None
logits = None
predicted_class_id = None
results_df = pd.DataFrame()

def infer(slide):
    global logits, outputs, predicted_class_id, probs
    image = Image.open(slide)
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = logits.argmax(-1).item()
    probs = torch.nn.functional.softmax(logits, dim=-1)
    print("Predicted class:", model.config.id2label[predicted_class_id])
    # print(f"Predicted class: {predicted_class_id}")
    

imageList = os.listdir('./datasets/brca-psj-path/contest-phase-2/clam-preprocessing-holdout/stitches')
       
for image in imageList[12000:]:    
    infer(f"./holdout/{image}")
    results_df.loc[imageList.index(image),0] = str(image[:-4]) # biopsy id
    results_df.loc[imageList.index(image),1] = float(probs[0][0].item()) # prob 0
    results_df.loc[imageList.index(image),2] = float(probs[0][1].item()) # prob I
    results_df.loc[imageList.index(image),3] = float(probs[0][2].item()) # prob II
    results_df.loc[imageList.index(image),4] = float(probs[0][3].item()) # prob III
    results_df.loc[imageList.index(image),5] = float(probs[0][4].item()) # prob IV
    results_df.loc[imageList.index(image),6] = int(predicted_class_id) # predicted class


# torch.nn.functional.softmax(logits, dim=-1)[0][0].item()

# Sample CSV
# 47ba1eb2-0d3b-4752-80d3-6d318001751e,0.18312,0.67326,0.11121,0.03231,8.1357e-05,1
# e4235769-c290-4bce-bf3a-9b98c7ef80b5,0.27895,0.25605,0.080242,0.38474,2.0360e-06,3
# d9bd5e69-98ce-4736-a108-fd64234ffb05,0.0064058,0.16384,0.33646,0.32201,0.17126,2

In [ ]:
# Final submission csv
df = results_df.rename(columns={'0': 'slide_id'})
df2 = pd.read_csv('datasets/brca-psj-path/contest-phase-2/slide-manifest-holdout.csv')
final_df = df.merge(df2, on='slide_id', how='left')
biopsies = final_df.drop_duplicates('biopsy_id')['biopsy_id'].to_list()
submit_df = pd.DataFrame(columns=['0','1','2','3','4','5','6'])

for biopsy in biopsies:
    row = (len(submit_df)+1)
    selection = final_df[final_df.biopsy_id==biopsy][['1','2','3','4','5']].mean()
    submit_df.loc[row,'0'] = str(biopsy)
    submit_df.loc[row,'1'] = float(selection[0])
    submit_df.loc[row,'2'] = float(selection[1])
    submit_df.loc[row,'3'] = float(selection[2])
    submit_df.loc[row,'4'] = float(selection[3])
    submit_df.loc[row,'5'] = float(selection[4])
    submit_df.loc[row,'6'] = int(list(selection).index(selection.max()))

# Change probability classification of class 0 breaat cancer (>15%)
submit_df.loc[submit_df['1']>0.15, '6'] = 0

In [ ]:
# Histoseg

import numpy as np
import os, cv2
from PIL import Image

height = 1000
width = 1000
channels = 3
dim = (width, height)
imageList = os.listdir('./slide_images/0')
x = np.ndarray(shape=(len(imageList[:10]),height,width,channels), dtype='uint8')


for i in range(len(imageList[:10])):
      # open image to numpy array
      img = Image.open(f"./slide_images/0/{imageList[i]}")
      img = img.resize((height,width))       
      # insert into placeholder array
      x[i] = np.array(img)

# for i in range(len(imageList[:10])):
#       # open image to numpy array
#       img = cv2.imread(f"./slide_images/0/{imageList[i]}")
#       # resize
#       img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
#       # insert into placeholder array
#       x[i] = img

np.save('dataset.npy', x)
    
# np.savez('dataset.npz', x=x)

# img2 = Image.fromarray(img2, 'RGB')
# img2.save('my.jpg')

In [ ]:
# !export TF_ENABLE_ONEDNN_OPTS=0
python HistoSeg_Test.py --images 'dataset.npz' --weights 'HistoSeg_MoNuSeg_.h5' --width 1000 --height 1000

In [ ]:
# EfficientViT 
python -m torch.distributed.launch --nproc_per_node=4 --master_port 12345 --use_env main.py --model EfficientViT_M5 --data-path /home/ngsci/stitch_images --dist-eval
/home/ngsci/slide_images

# InceptionResnetv2
python image_train.py 
    